In [ ]:
import requests
import os
import re
import time
import random
from urllib.request import urlopen
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select # Import Select class
from selenium.common.exceptions import TimeoutException, WebDriverException, NoSuchElementException
from selenium.webdriver.common.action_chains import ActionChains
from fake_useragent import UserAgent

base_url ='https://olimpiada.ic.unicamp.br/pratique/'
root = './sol'

def human_scroll(driver):
    scroll_height = driver.execute_script("return document.body.scrollHeight")
    for _ in range(random.randint(3, 6)):  
        scroll = random.randint(200, 600)  
        driver.execute_script(f"window.scrollBy(0, {scroll});")
        time.sleep(random.uniform(0.5, 1.5))  

def human_mouse_move(driver, element):
    actions = ActionChains(driver)
    actions.move_to_element(element).perform()
    
    time.sleep(random.uniform(0.5, 1.5))
def submit_file(base_url, file_path):
    file_path = os.path.abspath(file_path)
    time.sleep(random.uniform(5, 10))
    # Initialize UserAgent object
    ua = UserAgent()
    # Use selenium to handle delayed redirects
    options = webdriver.FirefoxOptions()
    options.set_preference("general.useragent.override", ua.random)
    #options.add_argument('--headless')  # Enabling headless mode
    driver = webdriver.Firefox(options=options)
    
     # Avoid Selenium detection
    options.set_preference("dom.webdriver.enabled", False)
    options.set_preference('useAutomationExtension', False)
    options.set_preference("general.platform.override", "Win32")  
    options.set_preference("privacy.trackingprotection.enabled", False)  

    try:
        driver.get(base_url)  # Navigate to the initial redirect URL
    except WebDriverException:
        print(f"{base_url} URL not found")
        driver.quit()
        return
    select_elem = None
    time.sleep(random.uniform(5, 10))
    
    # Extract CSRF token from cookies (some frameworks use cookies instead of hidden input)
    csrf_token = None
    for cookie in driver.get_cookies():
        if cookie['name'] == 'csrftoken':  # Django uses "csrftoken", other frameworks may differ
            csrf_token = cookie['value']
            break

    # If not found in cookies, look for the hidden form field
    if csrf_token is None:
        try:
            csrf_token = driver.find_element(By.NAME, 'csrfmiddlewaretoken').get_attribute('value')
        except NoSuchElementException:
            print("CSRF token not found")
            driver.quit()
            return
    
    human_scroll(driver)  # Scroll before submitting
    driver.execute_script("window.scrollTo(0, 0);")
    time.sleep(random.uniform(0.5, 1.5))
    
    try:
        select_elem = Select(driver.find_element(By.TAG_NAME, 'select'))
    except NoSuchElementException:
        if 'pu/' in base_url:
            print(f'{base_url} URL not found')
            return
        base_url=base_url.replace('ps/','pu/')
        submit_file(base_url, file_path)
        return
   
    file_input = driver.find_element(By.NAME, 'data')
    human_mouse_move(driver, file_input)
    time.sleep(random.uniform(1, 3))
    file_input.send_keys(file_path)
    time.sleep(random.uniform(1, 3))

    
    form_elem = driver.find_element(By.TAG_NAME, 'form')
    driver.execute_script("""
        var input = document.createElement('input');
        input.setAttribute('type', 'hidden');
        input.setAttribute('name', 'csrfmiddlewaretoken');
        input.setAttribute('value', arguments[0]);
        arguments[1].appendChild(input);
        return input;
    """, csrf_token, form_elem)
    
    time.sleep(random.uniform(2, 4))
    try:
        submit_button = form_elem.find_element(By.CSS_SELECTOR, "button[type='submit']")
        human_mouse_move(driver, submit_button)  # Move mouse to the button
        submit_button.click()
    except NoSuchElementException:
        print("Submit button not found, trying form.submit() instead.")
        form_elem.submit()
    # Wait for the final redirect (adjust timeout as needed)
    WebDriverWait(driver, 10).until(
        EC.url_contains('corrige')  # Wait for URL to change
    )
    inner_page_element = WebDriverWait(driver, 10).until(
        EC.presence_of_element_located((By.CLASS_NAME, "inner-page"))
    )
    inner_page_text = inner_page_element.get_attribute("innerText")
    if "Houve um erro durante o processamento: conexão com o servidor falhou, não retornou valor." in inner_page_text:
        print(f'Erro de conexão {base_url}.')
        driver.quit()
        return
    try:
        final_url = WebDriverWait(driver, 40).until(
            EC.url_contains('resultado')  # Wait for URL to change
        )
        center_element = WebDriverWait(driver, 10).until(
            EC.presence_of_element_located((By.CLASS_NAME, 'inner-page'))
        )
        human_scroll(driver)  # Scroll before submitting
        file_path = os.path.join('./sol', file_path.split('\\')[-1][:-4]) # Barra invertida para Windows
        file_path += '.txt'
        with open(os.path.join(file_path), "w", encoding='utf-8') as f:
            f.write(center_element.text.split('Tarefas Programação',1)[0])
        print(f"{file_path} File saved successfully.")
    except TimeoutException:
        print(f"{file_path} Timeout waiting for redirect.")

    driver.quit()  # Close the browser


In [9]:
import os
root = './pratique'
for folder in os.listdir(root):
    if not folder.startswith('2'):
        continue
    path = os.path.join(root,folder,folder+'.cpp')
    splitted =  folder.split('-')
    if len(splitted) != 4:
        print(f'{folder} inválido.')
        continue
    if os.path.exists(os.path.join('./sol',folder+'.txt')):
        continue
    edicao, nivel, fase, nome = splitted
    if os.path.exists(path):
        time.sleep(random.uniform(5, 10))
        submit_file(base_url+f'{nivel}/{edicao}/{fase}/{nome}/', path)

The geckodriver version (0.35.0) detected in PATH at C:\Users\analc\miniconda3\envs\envIC\Scripts\geckodriver.exe might not be compatible with the detected firefox version (135.0.0.2327); currently, geckodriver 0.36.0 is recommended for firefox 135.*, so it is advised to delete the driver in PATH and retry


Erro de conexão https://olimpiada.ic.unicamp.br/pratique/p1/2009/f1/overflow/.


The geckodriver version (0.35.0) detected in PATH at C:\Users\analc\miniconda3\envs\envIC\Scripts\geckodriver.exe might not be compatible with the detected firefox version (135.0.0.2327); currently, geckodriver 0.36.0 is recommended for firefox 135.*, so it is advised to delete the driver in PATH and retry


Erro de conexão https://olimpiada.ic.unicamp.br/pratique/p1/2009/f2/maratona/.


The geckodriver version (0.35.0) detected in PATH at C:\Users\analc\miniconda3\envs\envIC\Scripts\geckodriver.exe might not be compatible with the detected firefox version (135.0.0.2327); currently, geckodriver 0.36.0 is recommended for firefox 135.*, so it is advised to delete the driver in PATH and retry


Erro de conexão https://olimpiada.ic.unicamp.br/pratique/p2/2009/f2/banda/.


Exception ignored in: <function Service.__del__ at 0x00000177C39D7880>
Traceback (most recent call last):
  File "c:\Users\analc\miniconda3\envs\envIC\Lib\site-packages\selenium\webdriver\common\service.py", line 195, in __del__
    self.stop()
  File "c:\Users\analc\miniconda3\envs\envIC\Lib\site-packages\selenium\webdriver\common\service.py", line 152, in stop
    self.send_remote_shutdown_command()
  File "c:\Users\analc\miniconda3\envs\envIC\Lib\site-packages\selenium\webdriver\common\service.py", line 132, in send_remote_shutdown_command
    request.urlopen(f"{self.service_url}/shutdown")
  File "c:\Users\analc\miniconda3\envs\envIC\Lib\urllib\request.py", line 189, in urlopen
    return opener.open(url, data, timeout)
  File "c:\Users\analc\miniconda3\envs\envIC\Lib\urllib\request.py", line 489, in open
    response = self._open(req, data)
  File "c:\Users\analc\miniconda3\envs\envIC\Lib\urllib\request.py", line 506, in _open
    result = self._call_chain(self.handle_open, protoc

KeyboardInterrupt: 

In [ ]:
root = './sol'
for file in os.listdir(root):
    if file.endswith('.txt'):
      with open(os.path.join(root, file), "r", encoding='utf-8') as f:
        if 'Pontuação' not in f.read():
            print(f'{file} sem resultado.')